In [2]:
import json
import pandas as pd
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch, BootstrapFinetune
import os
from dotenv import load_dotenv
import pathlib
import re
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score
import dsp
import numpy as np
from scipy import sparse
import os
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_distances
import faiss
import json

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [28]:
path_env = pathlib.Path(os.getcwd()).parent.parent / '.env'
print(path_env)
load_dotenv(path_env)
api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = api_key

/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/.env


In [4]:
with open("train.json", 'r') as file:
    data = json.load(file)
    
with open("dev.json", 'r') as file:
    data2 = json.load(file)

In [16]:
others = [el for el in data + data2 if len(el["questions"])<=1 and len(el["questions"][0]["answers"])==2 ]

dataset = []
for el in others:
    #import pdb; pdb.set_trace()
    if el["label"] == "Supported" or el["label"] == "Refuted":
        label = "CONSISTENT"
    else:
        label = el["label"]
    if "claim" not in el["questions"][0]["question"]:
        this_entry = [
            #el["claim"], #claim
            el["questions"][0]["question"], # question
            el["questions"][0]["answers"][0]["answer"], #answer1
            el["questions"][0]["answers"][1]["answer"], #answer2
            label, # label
            el["justification"], #rationale
        ]
        dataset.append(this_entry)
    
df_dtset = pd.DataFrame(dataset, columns=["question", "answer1", "answer2", "label", "rationale"])
df_dtset[df_dtset.label!="CONSISTENT"]

,question,answer1,answer2,label,rationale
13,Is the Indian Government monitoring all forms of communication?,Debunked by alternate fact checking site.,"The Central Monitoring System (CMS), which was largely covered by the media in 2013, was actually approved by the Cabinet Committee on Security (CCS) on 16th June 2011",Conflicting Evidence/Cherrypicking,"Government of India monitors some forms of communications, but not all of them."
32,Can UK prisoners watch TV for free?,Prisoners must purchase phone credit themselves and pay rent for a TV.,Prisoners over retirement age do not have to pay for their TV.,Conflicting Evidence/Cherrypicking,"While it seems to be free for some prisoners, it is not free for all of them. therefore it is very hard to deduce from the the given question answers."
51,What percentage of kids in US schools speak Spanish?,The percentage of public school students in the United States who were English learners was 10.4% in the fall of 2019 and 9.2% in the fall of 2010.,About 6 in 10 young Hispanics say they use Spanish.,Not Enough Evidence,The first question and answer deals with English speakers not Spanish. The second question and answer is about Hispanic students not the general population in school so there is not enough evidence to see if this claim is true or not.


In [29]:
llm = dspy.OpenAI(
    model= "gpt-3.5-turbo",#"gpt-4o-2024-05-13" , #"gpt-4-0125-preview",  #gpt-4o-2024-05-13, #"gpt-4-1106-preview", # TODO: try turbo-instruct,
    max_tokens=1000)
dspy.settings.configure(lm=llm)

In [18]:
data_squad = "/export/usuarios_ml4ds/lbartolome/Repos/umd/LinQAForge/train-v2.0.json"
with open(data_squad, 'r') as file:
    data = json.load(file)

# Example: Flatten the JSON structure to create a DataFrame
# This example assumes you want to create a DataFrame from the 'data' field

data_flattened = []
for item in data['data']:
    title = item['title']
    for paragraph in item['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            qa_item = {
                'title': title,
                'context': context,
                'question': qa['question'],
                'is_impossible': qa['is_impossible'],
                'answers': qa['answers']
            }
            data_flattened.append(qa_item)

# Convert the flattened list to a DataFrame
samples_train = pd.DataFrame(data_flattened)

# Display the DataFrame
# [samples_train.is_impossible==True]
samples_train

,title,context,question,is_impossible,answers
0,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".",When did Beyonce start becoming popular?,False,"[{'text': 'in the late 1990s', 'answer_start': 269}]"
1,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".",What areas did Beyonce compete in when she was growing up?,False,"[{'text': 'singing and dancing', 'answer_start': 207}]"
2,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".",When did Beyonce leave Destiny's Child and become a solo singer?,False,"[{'text': '2003', 'answer_start': 526}]"
3,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".",In what city and state did Beyonce grow up?,False,"[{'text': 'Houston, Texas', 'answer_start': 166}]"
4,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singl

In [56]:
class Negator(dspy.Signature):
    """
    Generate an answer for the given question such that it contradicts the golden answer within the context of the question.

    Examples:
    --------
    QUESTION: "Can UK prisoners watch TV for free?"
    GOLDEN_ANSWER: "Prisoners must purchase phone credit themselves and pay rent for a TV."
    CHERRY_ANSWER: "Prisoners over retirement age do not have to pay for their TV."
    RATIONALE: "While it seems to be free for some prisoners, it is not free for all of them. therefore it is very hard to deduce from the the given question answers."
    """
    
    QUESTION = dspy.InputField()
    GOLDEN_ANSWER = dspy.InputField()
    CHERRY_ANSWER = dspy.OutputField()    
    RATIONALE = dspy.OutputField()    
    
class NegatorModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.negator = dspy.ChainOfThought(Negator)
    
    def forward(self, question, golden_answer):
        prediction = self.negator(QUESTION=question, GOLDEN_ANSWER=golden_answer)
        return dspy.Prediction(CHERRY_ANSWER=self.clean_cherry_answer(prediction.CHERRY_ANSWER), RATIONALE=prediction.RATIONALE)
    
    def clean_cherry_answer(self,cherry_answer):
        if "Reasoning:" in cherry_answer:
            cherry_answer = cherry_answer.split("Reasoning:")[0].strip()
        return cherry_answer
    
negator = NegatorModule()

In [57]:
example = samples_train.iloc[0]
print(example.question, example.answers[0]["text"])

negator(question=example.question, golden_answer=example.answers[0]["text"])

When did Beyonce start becoming popular? in the late 1990s


Prediction(
    CHERRY_ANSWER='in the early 2000s',
    RATIONALE="While Beyonce did achieve solo success in the early 2000s, her popularity began to rise in the late 1990s with Destiny's Child."
)

In [61]:
filtered_samples = samples_train[samples_train['answers'].apply(lambda x: len(x) > 0)]
df_unique_titles = filtered_samples.drop_duplicates(subset='title', keep='first')
df_unique_titles.reset_index(drop=True, inplace=True)

new_samples = []
for idx, el in df_unique_titles.iterrows():
    if idx % 100 == 0:
        print(f"-- -- Processing index {idx} out of {len(df_unique_titles)}")
    cherry_picker = negator(question=el.question, golden_answer=el.answers[0]["text"])
    new_samples.append(
        [
            el.question, # question
            el.answers[0]["text"], # answer1
            cherry_picker.CHERRY_ANSWER, # answer2
            "CONTRADICTION", # label
            cherry_picker.RATIONALE, # rationale
        ]
    )

# Convert new_samples to a DataFrame if needed
new_samples_df = pd.DataFrame(new_samples, columns=['question', 'answer1', 'answer2', 'label', 'rationale'])

-- -- Processing index 0 out of 440
-- -- Processing index 100 out of 440
-- -- Processing index 200 out of 440
-- -- Processing index 300 out of 440
-- -- Processing index 400 out of 440


In [64]:
new_samples_df.to_csv("ALL_CONTRAD_squad.csv")

In [66]:
new_samples_df_samp = new_samples_df.sample(n=len(df_dtset), random_state=1)

merged_df = pd.concat([new_samples_df_samp, df_dtset[df_dtset.label=="CONSISTENT"]], ignore_index=True)
merged_df.to_json('merged_file.json', orient='records', lines=True)

In [67]:
merged_df.label.unique()

array(['CONTRADICTION', 'CONSISTENT'], dtype=object)